In [1]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time

In [15]:
def marching_cube(thres,cells):
	# vertices use dictionary to avoid duplicate axes
	vertex_array = {}
	face_array = []
	t1 = time.time()
	# -------------------TODO------------------ 
	# compute vertices and faces
	# vertices: [N, 3]
	# faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
	# for-loop is allowed to reduce difficulty
    # -------------------TODO------------------ 
    
	def interpolate(p1, p2, valp1, valp2, iso=0.0):
		p1 = p1.astype(np.float64)
		p2 = p2.astype(np.float64)
		valp1 = valp1.astype(np.float64)
		valp2 = valp2.astype(np.float64)
		iso = np.array(iso, dtype=np.float64)
		t = (iso - valp1) / (valp2 - valp1 + 1e-8)
		return p1 + t * (p2 - p1)

	nx, ny, nz = cells.shape
	num_vertices = 0

	for x in range(nx - 1):
		for y in range(ny - 1):
			for z in range(nz - 1):
				cube_pos = np.array([x, y, z], dtype=np.float64)

    			# Calculate cube index
				intersect_edges = getCaseNum(x,y,z,thres,cells)

                # Interpolate edge vertices
				edge_vertices = [None] * 12
				for e in intersect_edges:
					if e == -1:
						break
					if edge_vertices[e] is not None:
						continue

					offset = CaseNum2EdgeOffset[e]
					v1 = (cube_pos + offset[:3]).astype(np.int32)
					v2 = (cube_pos + offset[3:]).astype(np.int32)
					val1 = cells[tuple(v1)]
					val2 = cells[tuple(v2)]

					v = interpolate(v1, v2, val1, val2, iso=thres).astype(np.float64)

					if vertex_array.get(tuple(v), None) is None:
						vertex_array[tuple(v)] = num_vertices
						edge_vertices[e] = num_vertices
						num_vertices += 1
					else:
						edge_vertices[e] = vertex_array[tuple(v)]

                # Create triangles
				for i in range(0, len(intersect_edges)-2, 3):
					e1, e2, e3 = intersect_edges[i:i+3]
					if e1 == -1:
						break
					face_array.append([edge_vertices[e1], edge_vertices[e2], edge_vertices[e3]])

	t2 = time.time()
	print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
	vertex_array = list(vertex_array.keys())
	return np.array(vertex_array), np.array(face_array)

In [16]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)


Time taken by algorithm
----------------------------------------
3.7090389728546143 s

Time taken by algorithm
----------------------------------------
3.856362819671631 s
